In [25]:
import torch
from tqdm import tqdm

from data.dataset import SkinDataset, NonSkinDataset
from settings import *
from utils.data_augmentation import *
from torch.utils.data import DataLoader, ConcatDataset
import torch.nn as nn
import torch.optim as optim

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [27]:
skin_file_paths = collect_file_paths(os.path.join(DATA_DIR, "SKIN"))
not_skin_file_paths = collect_file_paths(os.path.join(DATA_DIR, "NS"))

In [28]:
torch.manual_seed(42)

skin_indices = torch.randperm(len(skin_file_paths))
skin_file_paths = [skin_file_paths[i] for i in skin_indices]

not_skin_indices = torch.randperm(len(not_skin_file_paths))
not_skin_file_paths = [not_skin_file_paths[i] for i in not_skin_indices]

In [29]:
train_skin_file_paths = skin_file_paths[:NUM_TRAIN_SKIN]
train_not_skin_file_paths = not_skin_file_paths[:NUM_TRAIN_NOT_SKIN]

test_skin_file_paths = skin_file_paths[NUM_TRAIN_SKIN:NUM_TRAIN_SKIN + NUM_TEST_SKIN]
test_not_skin_file_paths = not_skin_file_paths[NUM_TRAIN_NOT_SKIN:
                                               NUM_TRAIN_NOT_SKIN + NUM_TEST_NOT_SKIN]

val_skin_file_paths = skin_file_paths[NUM_TRAIN_SKIN + NUM_TEST_SKIN:]
val_not_skin_file_paths = not_skin_file_paths[NUM_TRAIN_NOT_SKIN +
                                              NUM_TEST_NOT_SKIN:]

In [30]:
train_skin_dataset = SkinDataset(train_skin_file_paths)
train_not_skin_dataset = NonSkinDataset(train_not_skin_file_paths)

test_skin_dataset = SkinDataset(test_skin_file_paths)
test_not_skin_dataset = NonSkinDataset(test_not_skin_file_paths)

val_skin_dataset = SkinDataset(val_skin_file_paths)
val_not_skin_dataset = NonSkinDataset(val_not_skin_file_paths)

In [31]:
train_dataset = ConcatDataset([train_skin_dataset, train_not_skin_dataset])
test_dataset = ConcatDataset([test_skin_dataset, test_not_skin_dataset])
val_dataset = ConcatDataset([val_skin_dataset, val_not_skin_dataset])

In [32]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [33]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=3, out_channels=16, kernel_size=1, stride=1)
        self.conv2 = nn.Conv2d(
            in_channels=16, out_channels=32, kernel_size=2, stride=1)
        self.conv3 = nn.Conv2d(
            in_channels=32, out_channels=64, kernel_size=1, stride=1)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 17 * 17, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        # print("After conv1", x.shape)
        x = self.relu(self.conv2(x))
        # print("After conv2", x.shape)
        x = self.relu(self.conv3(x))
        # print("After conv3", x.shape)
        x = self.maxpool(x)
        # print("After maxpool2d", x.shape)
        x = self.flatten(x)
        # print("After flatten", x.shape)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        # print("After fc1", x.shape)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        # print("After fc2", x.shape)
        x = torch.sigmoid(self.fc3(x))
        return x

In [34]:
def train_cnn(train_loader, val_loader, num_epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, save_path='model/model.pth'):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = CNN().to(device)
    criterion = nn.BCELoss()
    optimizer = optim.RMSprop(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0

        # Create a tqdm progress bar for training
        with tqdm(total=len(train_loader), desc=f"Epoch {epoch+1}/{num_epochs}", unit='batch') as pbar:
            for _, (inputs, targets) in enumerate(train_loader):
                inputs, targets = inputs.to(device), targets.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                targets = targets.float().view(-1, 1)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

                pbar.set_postfix(loss=loss.item())
                pbar.update(1)

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}] - Average Training Loss: {avg_loss:.4f}")

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            # Create a tqdm progress bar for validation
            with tqdm(total=len(val_loader), desc=f"Validation Epoch {epoch+1}/{num_epochs}", unit='batch') as pbar:
                for val_inputs, val_targets in val_loader:
                    val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                    val_outputs = model(val_inputs)
                    val_loss += criterion(val_outputs, val_targets).item()
                    pbar.update(1)

        avg_val_loss = val_loss / len(val_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}] - Validation Loss: {avg_val_loss:.4f}")

        # Save the model after each epoch
        torch.save(model.state_dict(), save_path)
        print(f"Model saved to {save_path}")

    print("Training complete!")

In [35]:
# def train_cnn(train_loader, val_loader, num_epochs=NUM_EPOCHS, batch_size=BATCH_SIZE):
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model = CNN().to(device)
#     criterion = nn.BCELoss()
#     optimizer = optim.RMSprop(model.parameters(), lr=0.001)

#     for epoch in range(num_epochs):
#         model.train()
#         total_loss = 0.0
#         for batch_idx, (inputs, targets) in enumerate(train_loader):
#             inputs, targets = inputs.to(device), targets.to(device)
#             optimizer.zero_grad()
#             outputs = model(inputs)
#             targets = targets.float().view(-1, 1)
#             print(targets.shape, outputs.shape)
#             loss = criterion(outputs, targets)
#             loss.backward()
#             optimizer.step()
#             total_loss += loss.item()

#         avg_loss = total_loss / len(train_loader)
#         print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {avg_loss:.4f}")

#         model.eval()
#         with torch.no_grad():
#             val_loss = 0.0
#             for val_inputs, val_targets in val_loader:
#                 val_inputs, val_targets = val_inputs.to(
#                     device), val_targets.to(device)
#                 val_outputs = model(val_inputs)
#                 val_loss += criterion(val_outputs, val_targets).item()

#             avg_val_loss = val_loss / len(val_loader)
#             print(f"Validation Loss: {avg_val_loss:.4f}")

#     print("Training complete!")

In [ ]:
train_cnn(train_loader=train_loader, val_loader=val_loader,
          batch_size=BATCH_SIZE, num_epochs=NUM_EPOCHS)

Epoch 1/100:  82%|████████▏ | 804/983 [1:17:59<49:21, 16.54s/batch, loss=0.189]    